# 使用OpenAI工具进行提取

进行提取工作从未如此简单！OpenAI的工具调用能力是一个完美的选择，它可以从文本中提取多个不同类型的元素。

1106之后的模型使用了工具和支持"并行函数调用"，这使得提取工作变得非常容易。

In [8]:
from typing import List, Optional

from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain_openai import ChatOpenAI

In [2]:
# 确保使用支持工具的最新模型
model = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [3]:
# Pydantic是一种定义模式的简单方法
class Person(BaseModel):
    """用于提取人员信息的类。"""

    name: str  # 姓名
    age: Optional[int] = None  # 年龄（可选，默认值为None）

In [10]:
# 创建提取链，并将其赋值给变量chain
chain = create_extraction_chain_pydantic(Person, model)

In [11]:
# 使用chain对象调用invoke方法，传入一个包含输入信息的字典
chain.invoke({"input": "jane is 2 and bob is 3"})

[Person(name='jane', age=2), Person(name='bob', age=3)]

In [12]:
# 让我们定义另一个元素
class Class(BaseModel):
    """Information about classes to extract."""  # 关于要提取的课程信息

    teacher: str  # 教师姓名
    students: List[str]  # 学生姓名列表

In [13]:
# 创建一个基于Pydantic的提取链
chain = create_extraction_chain_pydantic([Person, Class], model)

In [14]:
chain.invoke({"input": "jane is 2 and bob is 3 and they are in Mrs Sampson's class"})

[Person(name='jane', age=2),
 Person(name='bob', age=3),
 Class(teacher='Mrs Sampson', students=['jane', 'bob'])]

## 在幕后

在幕后，这是一个简单的链条：

```python
from typing import Union, List, Type, Optional

from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool
from langchain_core.runnables import Runnable
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.language_models import BaseLanguageModel

_EXTRACTION_TEMPLATE = """提取并保存以下段落中提到的相关实体及其属性。

如果属性不存在且在函数参数中不是必需的，请不要在输出中包含它。"""

def create_extraction_chain_pydantic(
    pydantic_schemas: Union[List[Type[BaseModel]], Type[BaseModel]],
    llm: BaseLanguageModel,
    system_message: str = _EXTRACTION_TEMPLATE,
) -> Runnable:
    if not isinstance(pydantic_schemas, list):
        pydantic_schemas = [pydantic_schemas]
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", "{input}")
    ])
    tools = [convert_pydantic_to_openai_tool(p) for p in pydantic_schemas]
    model = llm.bind(tools=tools)
    chain = prompt | model | PydanticToolsParser(tools=pydantic_schemas)
    return chain
```